In [ ]:
import pandas as pd
path = r"D:\BONEYS\WEB\PYTHON\Project\Ola_Ride_Insights\OLA_DataSet.xlsx"
data = pd.read_excel(path)
df = pd.DataFrame(data)
df

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
#filling null values
df = df.bfill()
df = df.ffill()
#doing label encoding and pd.dummy
from sklearn.preprocessing import LabelEncoder
encoder_columns = ['Booking_Status','Incomplete_Rides','Ride_Distance','Driver_Ratings','Customer_Rating','Date','Time',
                   'Booking_ID','Customer_ID','Pickup_Location','Drop_Location',]
dummy_columns = ['Vehicle_Type','Canceled_Rides_by_Customer','Canceled_Rides_by_Driver','Incomplete_Rides_Reason','Payment_Method']
encoder = LabelEncoder()
for i in encoder_columns:
    df[f'encoded_{i}'] = encoder.fit_transform(df[f'{i}'])
df = pd.get_dummies(df,columns=dummy_columns,dtype=int)
#removing outliners
df['Booking_Value_normalized'] = (df['Booking_Value']-df['Booking_Value'].mean())/df['Booking_Value'].std()
df['C_TAT_normalized'] = (df['C_TAT']-df['C_TAT'].mean())/df['C_TAT'].std()
df['V_TAT_normalized'] = (df['V_TAT']-df['V_TAT'].mean())/df['V_TAT'].std()
condition1 = df['Booking_Value_normalized']<=3
condition2 = df['Booking_Value_normalized']>=-3
condition3 = df['C_TAT_normalized']<=3
condition4 = df['C_TAT_normalized']>=-3
condition5 = df['V_TAT_normalized']<=3
condition6 = df['V_TAT_normalized']>=-3
df = df[(condition1 & condition2 & condition3 & condition4 & condition5 & condition6 )]
print(df)

In [ ]:
#storing above data in mysql
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    username = "xxx", #give your username
    password = "xxx", #give your password
    database = "xxx" #give database name which you want to use
)
cursor=connection.cursor()

In [21]:
query1 = """
    create table Ola(
    Date date,
    Time time,
    Booking_ID varchar(20),
    Booking_Status varchar(500),
    Customer_ID varchar(20),
    Vehicle_Type varchar(20),
    Pickup_Location varchar(50),
    Drop_Location varchar(50),
    C_TAT float,
    V_TAT float,
    Canceled_Rides_by_Customer varchar(100),
    Canceled_Rides_by_Driver  varchar(100),
    Incomplete_Rides varchar(6),
    Incomplete_Rides_Reason varchar(100),
    Booking_Value int,
    Payment_Method varchar(20),               
    Ride_Distance int,                 
    Driver_Ratings float,               
    Customer_Rating float
)
"""
cursor.execute(query1)

In [22]:
data = []
for index in df.index:
    row = df.loc[index]
    row = row.values
    row = row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],float(row[8]),float(row[9]),row[10],row[11],row[12],row[13],int(row[14]),row[15],int(row[16]),float(row[17]),float(row[18])
    data.append(row)
query2 = "insert into Ola values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [ ]:
#1 retreive all successful bookings
query3 = "select * from ola where Booking_Status = 'Success'"
cursor.execute(query3)
for i in cursor:
    print(i)

In [ ]:
#2. Find the average ride distance for each vehicle type:
query4 = "select avg(Ride_Distance) as average_ride_distance, Vehicle_Type from ola group by Vehicle_Type"
cursor.execute(query4)
for i in cursor:
    print(i)

In [ ]:
#3. Get the total number of cancelled rides by customers:
query5 = "select count(Canceled_Rides_by_Customer) as total_rides_cancelled from ola"
cursor.execute(query5)
for i in cursor:
    print(i)

In [ ]:
#4 List the top 5 customers who booked the highest number of rides
query6 = "select Customer_ID, count(Booking_ID) as Total_Booking from ola group by Customer_ID order by Total_Booking desc limit 5"
cursor.execute(query6)
for i in cursor:
    print(i)

In [ ]:
#5. Get the number of rides cancelled by drivers due to personal and car-related issues:
query7 = "select count(Canceled_Rides_by_Driver) as no_of_rides_cancelled_by_driver from ola where Canceled_Rides_by_Driver = 'Personal & Car related issue'"
cursor.execute(query7)
for i in cursor:
    print(i)

In [ ]:
#6. Find the maximum and minimum driver ratings for Prime Sedan bookings:
query8 = "select max(Driver_Ratings),min(Driver_Ratings),Vehicle_Type from ola where Vehicle_Type='Prime Sedan'"
cursor.execute(query8)
for i in cursor:
    print(i)

In [ ]:
#7. Retrieve all rides where payment was made using UPI:
query9 = "select Booking_ID,Customer_ID,Pickup_Location,Drop_Location,Date,Time,Booking_Value,Ride_Distance,Payment_Method from ola where Payment_Method='UPI'"
cursor.execute(query9)
for i in cursor:
    print(i)

In [ ]:
#8. Find the average customer rating per vehicle type:
query10 = "select avg(Customer_Rating) as Average_Customer_Rating, Vehicle_Type from ola group by Vehicle_Type"
cursor.execute(query10)
for i in cursor:
    print(i)

In [ ]:
#9. Calculate the total booking value of rides completed successfully:
query11 = "select sum(Booking_Value) as Total_Booking_Value,Booking_Status from ola where Booking_Status = 'Success'"
cursor.execute(query11)
for i in cursor:
    print(i)

In [ ]:
#10. List all incomplete rides along with the reason
query12 = "select Booking_ID,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,Incomplete_Rides,Incomplete_Rides_Reason from ola where Incomplete_Rides='Yes'"
cursor.execute(query12)
for i in cursor:
    print(i)

In [13]:
#exporting to excel sheet
df.to_excel('ola_cleaned.xlsx', index=False)

In [6]:
#exporting to .csv
df.to_csv('ola_cleaned.csv', index=False)